In [ ]:
# # # Unzipping the uploaded file
!unzip -q /content/EmotionData.zip -d /content/


[/content/EmotionData.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/EmotionData.zip or
        /content/EmotionData.zip.zip, and cannot find /content/EmotionData.zip.ZIP, period.


In [ ]:
# # # Check contents of the unzipped folder
!ls /content/EmotionData


## Importing Dependencies

In [ ]:
# Import necessary libraries

import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import classification_report, confusion_matrix



## The porject is based on 2 levels

- level 1 : To preprocess and train catagories (e.g Negative, Neutral and Positive
- level 2 : Further classify the emotions into the specific catagories

## Data Preprocessing of Level1 Catagories

 - Negative
 - Neutral
 - Positive

In [ ]:
# Define base directory path
base_dir = '/content/EmotionData/'

# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Training set
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Validation set
)


Found 22968 images belonging to 3 classes.
Found 5741 images belonging to 3 classes.


## Build & Train the Model for Catagories

In [ ]:
# Define the CNN model for Level 1 classification
model_level1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model_level1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

# Train the model
history_level1 = model_level1.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


718/718 ━━━━━━━━━━━━━━━━━━━━ 614s 851ms/step - accuracy: 0.4797 - loss: 1.0828 - val_accuracy: 0.5731 - val_loss: 0.9274
Epoch 2/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 627s 858ms/step - accuracy: 0.5963 - loss: 0.8725 - val_accuracy: 0.5861 - val_loss: 0.8830
Epoch 3/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 601s 837ms/step - accuracy: 0.6628 - loss: 0.7529 - val_accuracy: 0.6231 - val_loss: 0.8354
Epoch 4/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 596s 830ms/step - accuracy: 0.7291 - loss: 0.6263 - val_accuracy: 0.6224 - val_loss: 0.8831
Epoch 5/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 596s 830ms/step - accuracy: 0.8188 - loss: 0.4453 - val_accuracy: 0.6041 - val_loss: 1.0765


## Evaluate the Model


In [ ]:
 # Evaluate model performance
val_loss, val_acc = model_level1.evaluate(validation_generator)
print(f'Validation Accuracy: {val_acc * 100:.2f}%')

180/180 ━━━━━━━━━━━━━━━━━━━━ 41s 230ms/step - accuracy: 0.6084 - loss: 1.0863
Validation Accuracy: 60.41%


In [ ]:
# Display confusion matrix and classification report
Y_pred = model_level1.predict(validation_generator)
Y_pred


180/180 ━━━━━━━━━━━━━━━━━━━━ 42s 230ms/step


array([[8.6699259e-01, 5.5970624e-03, 1.2741011e-01],
       [8.6116290e-01, 6.6049121e-02, 7.2787926e-02],
       [5.3615016e-01, 2.9819025e-02, 4.3403068e-01],
       ...,
       [2.7150540e-03, 9.2288232e-05, 9.9719268e-01],
       [5.2724892e-01, 2.3898438e-01, 2.3376665e-01],
       [6.8142807e-01, 3.1811678e-01, 4.5512052e-04]], dtype=float32)

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

print('Classification Report')
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[1402  402  867]
 [ 521  159  313]
 [1067  319  691]]
Classification Report
              precision    recall  f1-score   support

    Negative       0.47      0.52      0.50      2671
     Neutral       0.18      0.16      0.17       993
    Positive       0.37      0.33      0.35      2077

    accuracy                           0.39      5741
   macro avg       0.34      0.34      0.34      5741
weighted avg       0.38      0.39      0.39      5741



## Data Preprocessing for Level 2

 - Subcatagories of Level1

In [ ]:
# Define paths to Level 2 Positive dataset
positive_dir = '/content/EmotionData/Positive/'

train_datagen_positive = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator_positive = train_datagen_positive.flow_from_directory(
    positive_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_positive = train_datagen_positive.flow_from_directory(
    positive_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 8309 images belonging to 2 classes.
Found 2077 images belonging to 2 classes.


In [ ]:
# Define paths to Level 2 Negative dataset
negative_dir = '/content/EmotionData/Negative/'

train_datagen_negative = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator_negative = train_datagen_negative.flow_from_directory(
    negative_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_negative = train_datagen_negative.flow_from_directory(
    negative_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 10687 images belonging to 4 classes.
Found 2671 images belonging to 4 classes.


## Build and Train the Level 2 Models


In [ ]:
# Define the CNN model for Positive Emotion Classification (Level 2)
model_positive = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model_positive.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_positive = model_positive.fit(
    train_generator_positive,
    validation_data=validation_generator_positive,
    epochs=5
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


260/260 ━━━━━━━━━━━━━━━━━━━━ 214s 818ms/step - accuracy: 0.7434 - loss: 0.5630 - val_accuracy: 0.8002 - val_loss: 0.4281
Epoch 2/5
260/260 ━━━━━━━━━━━━━━━━━━━━ 256s 796ms/step - accuracy: 0.8609 - loss: 0.3424 - val_accuracy: 0.8209 - val_loss: 0.4160
Epoch 3/5
260/260 ━━━━━━━━━━━━━━━━━━━━ 206s 793ms/step - accuracy: 0.8978 - loss: 0.2531 - val_accuracy: 0.8710 - val_loss: 0.3229
Epoch 4/5
260/260 ━━━━━━━━━━━━━━━━━━━━ 206s 793ms/step - accuracy: 0.9284 - loss: 0.1812 - val_accuracy: 0.8652 - val_loss: 0.3476
Epoch 5/5
260/260 ━━━━━━━━━━━━━━━━━━━━ 206s 793ms/step - accuracy: 0.9590 - loss: 0.1178 - val_accuracy: 0.8739 - val_loss: 0.4371


In [ ]:
# Define the CNN model for Negative Emotion Classification (Level 2)
model_negative = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
model_negative.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_negative = model_negative.fit(
    train_generator_negative,
    validation_data=validation_generator_negative,
    epochs=5
)


Epoch 1/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 268s 797ms/step - accuracy: 0.3667 - loss: 1.4393 - val_accuracy: 0.4478 - val_loss: 1.1590
Epoch 2/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 326s 809ms/step - accuracy: 0.4821 - loss: 1.1039 - val_accuracy: 0.4747 - val_loss: 1.1134
Epoch 3/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 321s 805ms/step - accuracy: 0.5600 - loss: 0.9829 - val_accuracy: 0.4714 - val_loss: 1.1332
Epoch 4/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 321s 803ms/step - accuracy: 0.6311 - loss: 0.8318 - val_accuracy: 0.4826 - val_loss: 1.1749
Epoch 5/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 268s 802ms/step - accuracy: 0.7535 - loss: 0.6113 - val_accuracy: 0.4788 - val_loss: 1.4586


In [ ]:
def hierarchical_emotion_classification(image):
    # Step 1: Level 1 classification
    level1_pred = model_level1.predict(image)
    level1_class = np.argmax(level1_pred)

    if level1_class == 2:  # Positive
        # Step 2: Classify into Happy or Surprised
        level2_pred = model_positive.predict(image)
        emotion = 'Happy' if np.argmax(level2_pred) == 0 else 'Surprised'

    elif level1_class == 0:  # Negative
        # Step 2: Classify into Angry, Sad, Disgusted, Fearful
        level2_pred = model_negative.predict(image)
        emotion_labels = ['Angry', 'Sad', 'Disgusted', 'Fearful']
        emotion = emotion_labels[np.argmax(level2_pred)]

    else:
        # Neutral emotion
        emotion = 'Neutral'

    return emotion


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the test image
def load_image(img_path):
    # Load the image with target size
    img = image.load_img(img_path, target_size=(128, 128))
    # Convert the image to an array
    img_array = image.img_to_array(img)
    # Reshape and normalize the image array
    img_array = np.expand_dims(img_array, axis=0)  # Shape becomes (1, 128, 128, 3)
    img_array /= 255.  # Rescale pixel values
    return img_array


In [ ]:
# List of test image paths
test_images = ['/content/EmotionData/Negative/Disgusted/im10.png', '/content/EmotionData/Negative/Fearful/im1000.png','/content/EmotionData/Negative/Sad/im1001.png','/content/EmotionData/Negative/Angry/im1002.png']

# Iterate over the test images
for img_path in test_images:
    # Load and preprocess the image
    img = load_image(img_path)

    # Predict emotion
    emotion = hierarchical_emotion_classification(img)

    # Print the result
    print(f'Image: {img_path} | Predicted Emotion: {emotion}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Image: /content/EmotionData/Negative/Disgusted/im10.png | Predicted Emotion: Angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: /content/EmotionData/Negative/Fearful/im1000.png | Predicted Emotion: Fearful
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: /content/EmotionData/Negative/Sad/im1001.png | Predicted Emotion: Disgusted
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: /content/EmotionData/Negative/Angry/im1002.png | Predicted Emotion: Neutral


In [ ]:
# After training the model for positive emotions (happy, surprised)
model_positive.save('positive_emotion_classifier.h5')
# After training the model for negative emotions (sad, angry, fearful, etc.)
model_negative.save('negative_emotion_classifier.h5')
#
model_level1.save('level1_emotion_classifier.h5')

In [ ]:
!pip install flask-ngrok


In [ ]:
import tensorflow as tf
from flask import Flask, request, jsonify
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
from flask_ngrok import run_with_ngrok  # Import ngrok integration

# Load the saved models (adjust paths as needed)
model_level1 = tf.keras.models.load_model('level1_emotion_classifier.h5')
model_positive = tf.keras.models.load_model('positive_emotion_classifier.h5')
model_negative = tf.keras.models.load_model('negative_emotion_classifier.h5')

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when running the app

# Preprocessing function for images
def preprocess_image(img):
    img = img.resize((224, 224))  # Resize to the input size expected by your model
    img = np.array(img)
    img = img.astype('float32') / 255.0  # Normalize image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Hierarchical emotion classification
def classify_emotion(image):
    # Step 1: Use level 1 classifier to distinguish between positive and negative
    level1_prediction = model_level1.predict(image)
    level1_label = np.argmax(level1_prediction)  # 0 for negative, 1 for positive

    if level1_label == 1:
        # Positive emotions (happy, surprised)
        prediction = model_positive.predict(image)
        labels = ['happy', 'surprised']
    else:
        # Negative emotions (sad, angry, fearful)
        prediction = model_negative.predict(image)
        labels = ['sad', 'angry', 'fearful']

    predicted_label = labels[np.argmax(prediction)]
    return predicted_label

# Endpoint to classify emotion
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    try:
        img = Image.open(file)
        processed_image = preprocess_image(img)

        # Classify emotion hierarchically
        predicted_emotion = classify_emotion(processed_image)

        return jsonify({'emotion': predicted_emotion})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Main function to run the Flask app
if __name__ == '__main__':
    app.run()